In [1]:
# Notebook for prediction and evaluation of multi-step forecasting ARTRFDC models

In [2]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [3]:
# symmetrical mean absolute percentage error
def smape(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [4]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

# during batch prediction, the SLDB identifier is obtained via Abseil Flags
sldb_id = 'CPE04115_H_kw_20201021084001_ARTRFDC_168'

In [5]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [6]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [7]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']

In [8]:
# use the time series identifier to obtain the SK-Learn scaler used on it
scaler = joblib.load('{}/{}/{}/scaler.save'.format(PROJECT_ROOT,
                                                    'timeseries',
                                                    ts_identifier))

print('Scaler loaded for time series {}'.format(ts_identifier))

Scaler loaded for time series CPE04115_H_kw_20201021084001


/home/developer/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
# pass all the code to a single notebook cell, then to a function, later...

In [334]:
# during batch prediction, the model identifier is obtained via Abseil Flags
model_id = 'ARTRFDC_TPU_001'

In [335]:
# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

In [336]:
# during batch prediction, the execution identifier is obtained via Abseil Flags
execution = 1

In [337]:
# use model identifier and execution number to build the model directory string
model_dir = '{}_{:02d}'.format(model_id, execution)

In [338]:
# get the path to the saved model main directory
saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                     'models',
                                                     model_dir)

In [339]:
# get all the files in the saved model path, to find the most recent one
all_files = os.listdir(saved_model_path)
# get the path to the most recent saved model
latest_saved_model_id = sorted(all_files)[-1]

In [340]:
# build the full path for the latest saved model dir
export_dir = '{}/{}'.format(saved_model_path, latest_saved_model_id)
print ('Exported model path is {}'.format(export_dir))

Exported model path is /home/developer/gcp/cbidmltsf/models/ARTRFDC_TPU_001_01/export/exporter/1621561126


In [341]:
# load the saved model and the prediction function
imported = load(export_dir=export_dir, tags='serve')
predict_fn = imported.signatures["serving_default"]

In [342]:
# build a path to the dataset for prediction
dataset_path = '{}/{}.tfrecord'.format(data_dir, dataset)

# load the dataset
tfrecord_dataset = tf.data.TFRecordDataset(dataset_path)

In [343]:
# get the SLDB parameters for the forecasting model
config_json_file = '{}/{}/{}.json'.format(PROJECT_ROOT,
                                          'parameters',
                                          model_id)

In [344]:
# recover the sldb dictionary from the json file in parameters/
with open(config_json_file, 'r') as inputfile:
    configuration = json.load(inputfile)

In [345]:
# store the objective shapes for reshaping tensors in a dictionary
_EXTRACTING_OBJECTIVE_SHAPES = {
    'source': [configuration['num_timesteps'], configuration['model_dimension']],
    'target': [configuration['num_timesteps'], configuration['model_dimension']]
}

_EXTRACTING_OBJECTIVE_SHAPES

{'source': [168, 7], 'target': [168, 7]}

In [346]:
# this code block will be imported as the _parse_dataset_function

read_features = {
    'source': tf.io.VarLenFeature(dtype=tf.float32),
    'target': tf.io.VarLenFeature(dtype=tf.float32)
}


def _parse_dataset_function(example_proto, objective_shapes, parse_timestamp):
    # parse the input tf.Example proto using the dictionary above
    row = tf.io.parse_single_example(example_proto, read_features)
    
    # pass objective shape as a list of lists [hourly_shape, daily_shape, weekly_shape]
    source = tf.reshape(row['source'].values, objective_shapes['source'])
    target = tf.reshape(row['target'].values, objective_shapes['target'])

    # the parsed dataset must have the shape {features}, target!!!
    # so:
    feature_dict = {
        'source': source
    }
    
    # Do not parse the timestamp for training!!! Strings are not supported in TPUs!!!,
    # or parse it as a number
    if parse_timestamp:
        feature_dict['timestamp'] = timestamp

    # _parse_dataset_function returns:
    # features as a dictionary, and
    # target as a float vector
    return feature_dict, target

In [347]:
# parse the test dataset from the TFRecord file
# and use its features to produce predictions in a different way

parsed_dataset = tfrecord_dataset.map(
    lambda row: _parse_dataset_function(
        example_proto=row,
        objective_shapes=_EXTRACTING_OBJECTIVE_SHAPES,
        # ToDo: parse the timestamps for plotting or additional positional encoding, later...
        parse_timestamp=False
    )
)

In [348]:
# from now on, inferences for ARTRFDC are produced in a very different way
# from the used for DMSLSTM or EDALSTM models (prediction process has to be iterative)

In [349]:
# each predicted row is a sequence of n_timesteps values,
# but only the first element in this sequence is used, as the first prediction,
# then it is added (along with its positional encodings) to the end of the input sequence
# (first entry of the input sequence is discarded to keep tensor shape)
# to get the second prediction, and so on up to the n_timesteps-th prediction,
# which completes the n_timesteps prediction sequence (the forecast window)
# that starts immediately after the source input sequence ends (in time dimension)

In [350]:
# the iterative process for inference over the ARTRFDC saved model can be initiated now:
# source feature (?, 168, 7) (unseen data) is on parsed_dataset[0]['source']
# target feature (?, 168, 7) (unseen data) is on parsed_dataset[1]

# uncomment and run the following two cells to confirm that

In [351]:
# it is not possible to iterate over a segment of a dataset, as required by iterative inference
# then the complete test dataset will be passed to two NumPy arrays:

# source_array, with shape (n_rows, n_timesteps, n_features), in this example (2095, 168, 7), and
# target_array, with shape (n_rows, n_timesteps, n_features), in this example (2095, 168, 7)

# remember source_array[1:, :, :] = target_array[:-2, :, :]

In [352]:
source_list = list()
target_list = list()

for element in parsed_dataset:
    source_list.append(element[0]['source'])
    target_list.append(element[1])

source_array = np.array(source_list)
target_array = np.array(target_list)

In [353]:
# verify shape of resulting arrays
source_array.shape, target_array.shape

((2095, 168, 7), (2095, 168, 7))

In [354]:
# now follow the inference process detailed in Klingenbrunn to:
# predict over the forecast window,
# calculate prediction error metrics, and
# plot prediction results

In [355]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

In [356]:
# the first source or input to the model is the first source row
# that means, the true variable value, plus the six positional encodings for the timestamp
# in the first row of the test dataset

In [357]:
# uncomment the following line to get the source as a tensor with TensorShape([1, 168, 7])
# source_tensor = tf.expand_dims(source_array[0, :, :], axis=0)

In [358]:
# ToDo: include timestamps in train, eval, and test datasets to easily keep tracking of prediction dates!

In [359]:
# now please code all the complicated previous stuff into an easy Python function, would you?

In [360]:
# important, the inference cycle was coded for tensors, not for NumPy arrays
# then use source and prediction tensors and translate to tensor examples from float tensors

In [457]:
# which row of the test dataset will be used for inference?
row = 168

In [458]:
# then again, build the initial source tensor
source_tensor = tf.expand_dims(source_array[row, :, :], axis=0)
source_tensor.shape

TensorShape([1, 168, 7])

In [459]:
# and build the initial prediction tensor
# a forecast-window-sized tensor (1, forecast_window, 7)
# formed with the forecast_window true values, starting at the end of the source tensor
# that means

prediction_tensor = tf.expand_dims(source_array[row + 168, :forecast_window, :], axis=0)
prediction_tensor.shape

TensorShape([1, 24, 7])

In [460]:
# ToDo: find out how many prediction tensors can be formed from a dataset with n_rows!
# use this value as a limit for the inference process!

In [461]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [462]:
def tensor_to_tensor_example(float_tensor):
    # first, pass the float tensor to NumPy array, then flatten it
    flat_array = float_tensor.numpy().flatten()
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # individual feature definition
            feature={'source': _float_feature_from_list_of_values(flat_array)}
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    tensor_example = tf.convert_to_tensor(numpy_example)

    return tensor_example

In [463]:
next_input_model = source_tensor

In [464]:
# re-initialize the prediction list previously used for prediction over TFRecords
predictions_list = list()

In [465]:
for i in range(forecast_window):
    
    # from the current next_input_model tensor (1, 168, 7)
    # get a prediction as NumPy array (1, 168, 1)
    prediction = predict_fn(tensor_to_tensor_example(next_input_model))['forecast'].numpy()
    
    # get the value of the most recent prediction (last timestep) into the predictions list
    predictions_list.append(prediction[:, -1, :][0][0])
    
    # from the source tensor, get the positional encodings for ti+1 to t167 (that is 168-i-1 values)
    pos_encoding_old_values = source_tensor[:, i+1:, 1:]

    # from target tensor, get the positional encodings for t168 to t168+i (that is i+1 values)
    pos_encoding_new_val = prediction_tensor[:, :i+1, 1:]

    # build new positional encodings with 168 values
    pos_encodings = tf.concat([pos_encoding_old_values, pos_encoding_new_val], axis=1)
    pos_encodings = tf.cast(pos_encodings, dtype=tf.float32)

    # build the values feature for the next input to the model
    # pop i+1 values at the beginning of the previous input
    value_feature_old_values = tf.expand_dims(source_tensor[:, i+1:, 0], axis=-1)
    value_feature_old_values = tf.cast(value_feature_old_values, dtype=tf.float32)

    # current predictions_list to NumPy array
    value_feature_new_values = np.array(predictions_list[:i+1])

    # current prediction array to tensor
    value_feature_new_values = tf.convert_to_tensor(value_feature_new_values)

    # expand dimensions of current prediction tensor to single-value feature
    value_feature_new_values = tf.expand_dims(value_feature_new_values, axis=-1)

    # expand dimensions of current prediction tensor to single-value batch
    value_feature_new_values = tf.expand_dims(value_feature_new_values, axis=0)
    
    # build the value feature tensor
    next_input_model = tf.concat([value_feature_old_values, value_feature_new_values], axis=1)
    
    # build the next input tensor for the model
    next_input_model = tf.concat([next_input_model, pos_encodings], axis=2)

In [466]:
# iterative predictions over the forecast window reside in predictions_list
np.array(predictions_list)

array([0.5235025 , 0.5317937 , 0.50272   , 0.5320462 , 0.5496116 ,
       0.4500504 , 0.33263466, 0.21609625, 0.14511836, 0.10028085,
       0.07016093, 0.06287903, 0.07648793, 0.16574663, 0.22860813,
       0.24710315, 0.3021697 , 0.38982624, 0.4534402 , 0.483442  ,
       0.5009064 , 0.5435877 , 0.55509764, 0.5044207 ], dtype=float32)

In [467]:
# and the true values remain in the prediction tensor
prediction_tensor[0, :, 0]

<tf.Tensor: shape=(24,), dtype=float32, numpy=
array([0.48842862, 0.4568397 , 0.4524253 , 0.5191421 , 0.55113226,
       0.48998427, 0.3736661 , 0.26310343, 0.18418387, 0.12973031,
       0.08332933, 0.09128969, 0.10174082, 0.12976441, 0.18660644,
       0.19949254, 0.24072066, 0.27756765, 0.33287108, 0.4307677 ,
       0.5224649 , 0.5580235 , 0.5968128 , 0.52098125], dtype=float32)>

In [468]:
smape(prediction_tensor[0, :, 0], np.array(predictions_list))

0.1618581215540568

In [384]:
# and the predictive performance of the ARTRFDC model is very sensitive to the input source!